# Traffic Simulation for Monitoring Dashboard

**Goal:** Send periodic inference requests to the deployed endpoint to simulate
real-world traffic patterns and populate CloudWatch metrics and Model Monitor data.

**Pre-requisite:** Run `2_deploy_and_monitor.ipynb` first to deploy the endpoint.

**Traffic Pattern:**
- Sends batches of predictions every 60 seconds
- Configurable duration (default: 30 minutes)
- Uses test dataset samples in random order
- Displays real-time progress and accuracy

**Use Case:**
- Populate CloudWatch dashboard with live metrics
- Generate data capture files for Model Monitor
- Demonstrate monitoring capabilities over time

## Setup

In [63]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sqlalchemy import create_engine
from sagemaker.predictor import Predictor
from sklearn.metrics import accuracy_score, roc_auc_score

sess   = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()

s3_prefix = "aai540/model/xgboost-binary"

FEATURE_COLS = [
    "duration", "pkt_total", "bytes_total",
    "pkt_fwd", "pkt_bwd", "bytes_fwd", "bytes_bwd",
    "pkt_rate", "byte_rate", "bytes_per_pkt",
    "pkt_ratio", "byte_ratio",
]
LABEL_COL = "label"

ENDPOINT_NAME = "ids-xgboost-binary-monitor"

print(f"Region   : {region}")
print(f"Endpoint : {ENDPOINT_NAME}")

Region   : us-east-1
Endpoint : ids-xgboost-binary-monitor


## Load Test Data

In [64]:
# Load test data from Athena
database_name = "aai540_eda"
engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={
        "s3_staging_dir": f"s3://{bucket}/athena/staging/",
        "region_name": region,
    },
)

columns = ", ".join([LABEL_COL] + FEATURE_COLS)
query = f"""
SELECT {columns}
FROM {database_name}.dataset_split
WHERE data_split = 'test'
"""
print("Loading test data from Athena...")
df_test = pd.read_sql(query, engine)
print(f"✓ Test rows loaded: {len(df_test):,}")

X_test = df_test[FEATURE_COLS].values
y_true = df_test[LABEL_COL].values

Loading test data from Athena...
✓ Test rows loaded: 49,920


## Connect to Endpoint

In [65]:
predictor = Predictor(
    endpoint_name=ENDPOINT_NAME,
    sagemaker_session=sess,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)
print(f"✓ Connected to endpoint: {ENDPOINT_NAME}")

✓ Connected to endpoint: ids-xgboost-binary-monitor


## Traffic Simulation Configuration

In [66]:
# Configuration
DURATION_MINUTES = 180       # Total simulation duration (3 hours)
INTERVAL_SECONDS = 60        # Time between batches
BATCH_SIZE = 100             # Samples per batch

total_iterations = (DURATION_MINUTES * 60) // INTERVAL_SECONDS

print("=" * 60)
print("  Traffic Simulation Configuration")
print("=" * 60)
print(f"  Duration        : {DURATION_MINUTES} minutes ({DURATION_MINUTES/60:.1f} hours)")
print(f"  Interval        : {INTERVAL_SECONDS} seconds between batches")
print(f"  Batch size      : {BATCH_SIZE} samples")
print(f"  Total batches   : {total_iterations}")
print(f"  Total requests  : {total_iterations * BATCH_SIZE:,}")
print("=" * 60)
print(f"\nStart time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Expected end: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} + {DURATION_MINUTES} min\n")

  Traffic Simulation Configuration
  Duration        : 180 minutes (3.0 hours)
  Interval        : 60 seconds between batches
  Batch size      : 100 samples
  Total batches   : 180
  Total requests  : 18,000

Start time: 2026-02-17 00:50:44
Expected end: 2026-02-17 00:50:44 + 180 min



## Run Simulation

This cell will run for the configured duration. You can interrupt it at any time with the stop button.

**Monitoring Tips:**
- Open the CloudWatch dashboard in another browser tab
- Metrics update every 1-5 minutes
- Data capture files appear in S3 within ~2 minutes
- Model Monitor runs on the next hourly schedule

In [ ]:
all_predictions = []
all_actuals = []
iteration = 0

try:
    for i in range(total_iterations):
        iteration = i + 1
        
        # Random sample from test set
        indices = np.random.choice(len(X_test), size=BATCH_SIZE, replace=False)
        batch_X = X_test[indices]
        batch_y = y_true[indices]
        
        # Send predictions
        start_time = time.time()
        response = predictor.predict(batch_X)
        latency = (time.time() - start_time) * 1000  # ms
        
        # Parse predictions
        y_prob = np.array([float(row[0]) for row in response])
        y_pred = (y_prob >= 0.5).astype(int)
        
        # Track results
        all_predictions.extend(y_pred)
        all_actuals.extend(batch_y)
        
        # Compute running metrics
        batch_accuracy = accuracy_score(batch_y, y_pred)
        cumulative_accuracy = accuracy_score(all_actuals, all_predictions)
        
        # Progress update
        timestamp = datetime.now().strftime('%H:%M:%S')
        elapsed = i * INTERVAL_SECONDS
        remaining = (total_iterations - i - 1) * INTERVAL_SECONDS
        
        print(
            f"[{timestamp}] Batch {iteration}/{total_iterations} | "
            f"Samples: {BATCH_SIZE} | Latency: {latency:.0f}ms | "
            f"Batch Acc: {batch_accuracy:.3f} | "
            f"Cumulative Acc: {cumulative_accuracy:.4f} | "
            f"Remaining: {remaining//60}m {remaining%60}s"
        )
        
        # Wait before next batch (unless last iteration)
        if i < total_iterations - 1:
            time.sleep(INTERVAL_SECONDS)
            
except KeyboardInterrupt:
    print(f"\n⚠️  Simulation interrupted at batch {iteration}/{total_iterations}")

# Final summary
print("\n" + "=" * 60)
print("  Simulation Complete")
print("=" * 60)
print(f"  Batches sent    : {iteration}/{total_iterations}")
print(f"  Total samples   : {len(all_predictions):,}")
print(f"  Overall accuracy: {accuracy_score(all_actuals, all_predictions):.4f}")
print(f"  End time        : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print("\n💡 Check your CloudWatch dashboard for updated metrics!")

[00:50:45] Batch 1/180 | Samples: 100 | Latency: 83ms | Batch Acc: 0.230 | Cumulative Acc: 0.2300 | Remaining: 179m 0s


## Quick Dashboard Link

In [ ]:
dashboard_url = (
    f"https://{region}.console.aws.amazon.com/cloudwatch/home"
    f"?region={region}#dashboards/dashboard/IDS-XGBoost-Monitoring"
)
print("CloudWatch Dashboard:")
print(dashboard_url)